In [6]:
import json
import os
import chromadb
from typing import Annotated, TYPE_CHECKING

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

if TYPE_CHECKING:
    from chromadb.api.models.Collection import Collection

In [7]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)


# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [13]:
import random
import numpy as np
import pandas as pd
import os
from typing import Annotated, Dict, Any, Optional
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

class BloodPressurePlugin:
    """Genera registros sintéticos de presión arterial basados en un DataFrame existente o de cero."""
    def __init__(self, data: Optional[pd.DataFrame] = None, random_state: int = 42):
        # Si no se provee data, inicializa vacío
        self.df = data.copy() if data is not None else pd.DataFrame()
        self._rng = np.random.default_rng(random_state)

    @kernel_function(description="Provee n registros sintéticos de presión arterial.")
    def generate_synthetic_data(
        self,
        n_samples: Annotated[int, "Número de muestras a generar"],
        output_file: Annotated[Optional[str], "Ruta CSV para guardar, opcional"] = None
    ) -> Annotated[pd.DataFrame, "DataFrame con datos sintéticos de presión arterial"]:
        """
        Genera un DataFrame con n_samples registros sintéticos.
        Si output_file es provisto, guarda el CSV.
        """
        # Parámetros base
        base_sys, base_dia = 120, 80
        rows = []

        for _ in range(n_samples):
            # Muestreo de fila base si existe df
            if not self.df.empty:
                row0 = self.df.sample(n=1, random_state=self._rng.bit_generator).iloc[0].to_dict()
            else:
                row0 = {}

            # Generación de variables demográficas y de salud
            age = max(18, min(90, self._rng.normal(45, 15)))
            gender = self._rng.choice(['Male', 'Female'])
            height_cm = float(
                self._rng.normal(175, 8) if gender=='Male' else self._rng.normal(162,7)
            )
            weight_kg = float(
                self._rng.normal(80, 15) if gender=='Male' else self._rng.normal(65,12)
            )
            bmi = weight_kg / ((height_cm/100) ** 2)
            smoking = self._rng.choice(['Never','Former','Current'], p=[0.6,0.2,0.2])
            smoking_num = {'Never':0,'Former':1,'Current':2}[smoking]
            exercise = float(self._rng.exponential(3))
            stress = int(self._rng.integers(1,11))
            heart_rate = float(self._rng.normal(75,10))
            cholesterol = float(self._rng.normal(200,40))
            glucose = float(self._rng.normal(90,15))
            diabetes = bool(self._rng.choice([0,1], p=[0.9,0.1]))
            fh_htn = bool(self._rng.choice([0,1], p=[0.7,0.3]))
            prev_cv = bool(self._rng.choice([0,1], p=[0.85,0.15]))
            on_med = bool(self._rng.choice([0,1], p=[0.8,0.2]))

            # Cálculo de efectos
            age_eff_s = 0.4 * (age-30)
            age_eff_d = 0.25 * (age-30)
            bmi_eff_s = 1.5 * (bmi-22.5)
            bmi_eff_d = 1.0 * (bmi-22.5)
            life_eff_s = 3*smoking_num - 0.5*exercise + 1.0*stress
            life_eff_d = 2*smoking_num - 0.3*exercise + 0.7*stress
            health_eff_s = (
                0.2*(heart_rate-75) + 0.05*(cholesterol-200) + 0.03*(glucose-90)
                + 5*diabetes + 4*fh_htn + 7*prev_cv
            )
            health_eff_d = (
                0.1*(heart_rate-75) + 0.03*(cholesterol-200) + 0.02*(glucose-90)
                + 3*diabetes + 2*fh_htn + 4*prev_cv
            )
            med_eff_s = -8 * on_med
            med_eff_d = -5 * on_med
            gender_eff_s = 2 if gender=='Male' else 0
            gender_eff_d = 1 if gender=='Male' else 0

            # Lecturas brutas
            systolic_bp = (
                base_sys + age_eff_s + bmi_eff_s + life_eff_s
                + health_eff_s + med_eff_s + gender_eff_s
                + self._rng.normal(0,8)
            )
            diastolic_bp = (
                base_dia + age_eff_d + bmi_eff_d + life_eff_d
                + health_eff_d + med_eff_d + gender_eff_d
                + self._rng.normal(0,5)
            )

            # Convertir a enteros
            systolic_bp = int(np.round(systolic_bp).clip(90,200))
            diastolic_bp = int(np.round(diastolic_bp).clip(50,120))

            # Añadir fila
            rows.append({
                'age': round(age,1),
                'gender': gender,
                'height_cm': round(height_cm,1),
                'weight_kg': round(weight_kg,1),
                'bmi': round(bmi,2),
                'smoking_status': smoking,
                'exercise_hours_per_week': round(exercise,1),
                'stress_level': stress,
                'heart_rate': round(heart_rate,1),
                'cholesterol': round(cholesterol,1),
                'glucose': round(glucose,1),
                'diabetes': diabetes,
                'family_history_hypertension': fh_htn,
                'previous_cardiovascular_condition': prev_cv,
                'on_medication': on_med,
                'systolic_bp': systolic_bp,
                'diastolic_bp': diastolic_bp
            })

        # Construir DataFrame
        data = pd.DataFrame(rows)

        # Guardar si se especifica output_file
        if output_file:
            os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)
            data.to_csv(output_file, index=False)

        return data

# Parte 2: Configuración del Agente
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

# Carga tu base de datos real si quieres usarla para muestreo
# df = pd.read_csv('blood_pressure_dataset.csv')
# bp_plugin = BloodPressurePlugin(data=df, random_state=123)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[# bp_plugin
    ],
    name="BPAgent",
    instructions=(
        "Eres un agente que genera registros sintéticos de presión arterial "
        "basados en una base de datos existente o aleatorios si no se provee."
    ),
)

# Parte 3: Flujo Principal
async def main():
    thread: ChatHistoryAgentThread | None = None
    user_inputs = [
        "Genera 100 registros sintéticos y guárdalos en data.csv",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(response, end="", flush=True)
            thread = response.thread
        print()

    await thread.delete() if thread else None

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [14]:
import asyncio
from typing import Annotated, Dict, Any, Optional
import pandas as pd
import numpy as np
import os

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

# Define tu plugin
class BloodPressurePlugin:
    def _init_(self, random_state: int = 42):
        self._rng = np.random.default_rng(random_state)

    @kernel_function(description="Genera un registro sintético de presión arterial.")
    def generate_synthetic_record(self) -> Annotated[Dict[str, Any], "Registro de presión arterial sintético"]:
        # Implementa la lógica para generar un registro sintético
        age = self._rng.integers(18, 90)
        systolic_bp = self._rng.integers(90, 180)
        diastolic_bp = self._rng.integers(60, 120)
        return {
            "age": age,
            "systolic_bp": systolic_bp,
            "diastolic_bp": diastolic_bp
        }

# Configura el kernel y el servicio de OpenAI
kernel = Kernel()
chat_service = OpenAIChatCompletion(
    service_id="openai-chat",
    api_key=os.getenv("OPENAI_API_KEY"),
    organization=os.getenv("OPENAI_ORG_ID")  # Si es necesario
)
kernel.add_service(chat_service)

# Registra el plugin
bp_plugin = BloodPressurePlugin()
kernel.add_plugin("BloodPressurePlugin", bp_plugin)

# Configura el agente
agent = ChatCompletionAgent(
    service=chat_service,
    plugins=[bp_plugin],
    name="BPAgent",
    instructions="Eres un agente que genera registros sintéticos de presión arterial."
)

# Función principal
async def main():
    thread = ChatHistoryAgentThread()
    user_input = "Genera un registro sintético de presión arterial."

    async for response in agent.invoke_stream(messages=user_input, thread=thread):
        print(response)

# Ejecuta la función principal
if _name_ == "_main_":
    asyncio.run(main())

TypeError: OpenAIChatCompletion.__init__() got an unexpected keyword argument 'organization'